In [1]:
import numpy as np
import re
from math import sqrt

def intersect(invindex,terms):
    d = {}
    for t in terms:
        d[t] = len(invindex[t])
    s = sorted(d)
    p1 = invindex[s[0]]
    for i in range(1,len(s)):
        p1 = intersectwithskips(p1,s[i])
    return p1
    
def intersectwithskips(p1,p2):
    l = []
    i = 0
    j = i
    lp1 = len(p1)
    lp2 = len(p2)
    while i<lp1 and j<lp2:
        if p1[i] == p2[j]:
            l.append(p1[i])
            i += 1
            j += 1
        elif p1[i] < p2[j]:
            skip = i+sqrt(lp1)
            if skip<lp1 and p1[skip]<=p2[i]:
                while skip<lp1 and p1[skip]<=p2[i]:
                    i = skip
                i += 1
            else:
                skip = j+sqrt(lp2)
                if skip<lp2 and p2[skip]<=p1[i]:
                    while skip<lp2 and p2[skip]<=p1[i]:
                        j = skip
                    j += 1
    return l
    
def cosScore(invindex,query,k):
    if '*' in query:
        terms = re.findall(query,' '.join(list(invindex.keys())))
        doclist = intersect(invindex,terms)
    else:
        N = 418
        scores = np.zeros(N)
        doclist = posintersect(invindex,query) #low idf will be removed
    qlist = query.split()
    qset = set(qlist)
    query_tf_vector = np.array([qlist.count(t) for t in qset])
    idf_vector = np.array([np.log(N//len(invindex[t])) for t in qset])
    w0 = query_tf_vector*idf_vector
    for docid in doclist:
        doc_tf_vector= np.array([len(invindex[t][docid]) for t in qset])
        w = doc_tf_vector*idf_vector
        scores[docid] = np.dot(w,w0)/(np.linalg.norm(w)*np.linalg.norm(w0))
    return np.sort(scores)[:len(doclist)]